# Atmo Model Training Notebook

Train an Atmo Model using `usl_models` lib.

In [ ]:
%load_ext autoreload
%autoreload 2
import logging
import os, time
import pathlib

import keras

from usl_models.atmo_ml.model import AtmoModel
from usl_models.atmo_ml import dataset, visualizer, vars


logging.getLogger().setLevel(logging.WARNING)
visualizer.init_plt()
keras.utils.set_random_seed(812)

batch_size = 4
filecache_path = pathlib.Path("/home/shared/climateiq/filecache")
example_keys=[
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-25"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-26"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-27"),
    ("NYC_Heat_Test/NYC_summer_2000_01p", "2000-05-28"),
]
ds_config = dataset.Config(
    input_width=200, input_height=200,
    output_width=200, output_height=200)
train_frac = 0.8
train_ds = dataset.load_dataset_cached(
    filecache_path,
    example_keys=example_keys,
    config=ds_config
).batch(batch_size=batch_size)
val_ds = dataset.load_dataset_cached(
    filecache_path,
    example_keys=example_keys,
    config=ds_config
).batch(batch_size=batch_size)

In [ ]:
lstm_dropout = 0.2
run_name = "46x46_clipnorm100x"
params = AtmoModel.dataset_config_params(ds_config)
params.update(
    {
        "lstm_dropout": lstm_dropout,
        "lstm_recurrent_dropout": lstm_dropout,
    }
)
model = AtmoModel(params=params)

log_dir = os.path.join("./logs", run_name)
print(log_dir)

tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir)
model.fit(train_ds, val_ds, epochs=10, callbacks=[tb_callback], validation_freq=5)
model.save_model(log_dir + "/model")

In [ ]:
# Visualize model outputs for the given variables.

st_var = vars.Spatiotemporal.RH
sto_var = vars.SpatiotemporalOutput.RH2

# To load a previous model:
# model = AtmoModel.from_checkpoint("logs/lstm_dropout_0.2/model")

for input_batch, label_batch in val_ds:
    pred_batch = model.call(input_batch)
    for b, _ in enumerate(label_batch):
        figs = visualizer.plot(
            inputs={k: v[b] for k, v in input_batch.items()},
            label=label_batch[b],
            pred=pred_batch[b],
            st_var=st_var,
            sto_var=sto_var,
        )
        for fig in figs:
            fig.show()